# Getting Started with pygSQuiG

Welcome to pygSQuiG! This notebook will help you:
1. Verify your installation
2. Create your first grid and solver
3. Run a simple turbulence simulation
4. Understand key diagnostics
5. Troubleshoot common issues

## 1. Installation Check

First, let's verify that pygSQuiG and its dependencies are properly installed:

In [ ]:
# Check imports
try:
    import jax
    import jax.numpy as jnp
    import numpy as np
    import matplotlib.pyplot as plt
    import pygsquig
    print("✓ All imports successful!")
    print(f"  JAX version: {jax.__version__}")
    print(f"  NumPy version: {np.__version__}")
    print(f"  Default device: {jax.default_backend()}")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please install pygSQuiG: pip install -e .")

## 2. Creating a Grid

pygSQuiG uses spectral methods on a doubly-periodic square domain [0, L] × [0, L]:

In [ ]:
from pygsquig.core.grid import make_grid

# Grid parameters
N = 128          # Number of grid points (use powers of 2 for FFT efficiency)
L = 2 * np.pi   # Domain size

# Create grid
grid = make_grid(N, L)

print(f"Grid properties:")
print(f"  Resolution: {grid.N} × {grid.N}")
print(f"  Domain size: {grid.L:.2f} × {grid.L:.2f}")
print(f"  Grid spacing: Δx = Δy = {grid.L/grid.N:.3f}")
print(f"  Nyquist wavenumber: k_max = {grid.N//2}")

## 3. Creating a Solver

The gSQG solver requires:
- **α**: Fractional power (α=1 for classic SQG)
- **Dissipation**: Small-scale dissipation to ensure numerical stability

In [ ]:
from pygsquig.core.solver import gSQGSolver

# Physical parameters
alpha = 1.0     # SQG turbulence
nu_p = 1e-16    # Hyperviscosity coefficient
p = 8           # Hyperviscosity order: dissipation ~ k^(2p)

# Create solver
solver = gSQGSolver(grid, alpha=alpha, nu_p=nu_p, p=p)

print(f"Solver configuration:")
print(f"  Model: gSQG with α = {alpha}")
print(f"  Dissipation: ν_{p} (-Δ)^{p/2} with ν_{p} = {nu_p:.1e}")

# Important note about hyperviscosity
print("\n⚠️  Important: For p=8 hyperviscosity, use ν_p ~ 1e-16")
print("   Using larger values (e.g., 1e-2) will cause immediate NaN!")

## 4. Running Your First Simulation

Let's simulate freely decaying turbulence from random initial conditions:

In [ ]:
# Initialize with random initial condition
state = solver.initialize(seed=42)

# The state contains:
print("State variables:")
for key, value in state.items():
    if hasattr(value, 'shape'):
        print(f"  {key}: shape {value.shape}, dtype {value.dtype}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# Time stepping
dt = 0.001      # Time step
n_steps = 100   # Number of steps

# Storage for diagnostics
from pygsquig.utils.diagnostics import compute_total_energy, compute_enstrophy

times = [state['time']]
energies = [compute_total_energy(state['theta_hat'], grid, alpha)]
enstrophies = [compute_enstrophy(state['theta_hat'], grid, alpha)]

print(f"Initial state:")
print(f"  Energy: {energies[0]:.6f}")
print(f"  Enstrophy: {enstrophies[0]:.3f}")

# Run simulation
print("\nRunning simulation...")
for i in range(n_steps):
    state = solver.step(state, dt)
    
    # Check for numerical issues
    theta_hat = state['theta_hat']
    if jnp.any(jnp.isnan(theta_hat)):
        print(f"❌ NaN detected at step {i+1}!")
        break
    
    # Record diagnostics every 10 steps
    if (i + 1) % 10 == 0:
        E = compute_total_energy(theta_hat, grid, alpha)
        Z = compute_enstrophy(theta_hat, grid, alpha)
        
        times.append(state['time'])
        energies.append(float(E))
        enstrophies.append(float(Z))
        
        if (i + 1) % 50 == 0:
            print(f"  Step {i+1}: t={state['time']:.3f}, E={E:.6f}")

print("\n✓ Simulation completed successfully!")

## 5. Visualizing Results

In [ ]:
# Convert to physical space for visualization
from pygsquig.core.grid import ifft2

theta = ifft2(state['theta_hat']).real

# Plot the field
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Buoyancy field
im = ax1.imshow(theta, cmap='RdBu_r', origin='lower',
                extent=[0, L, 0, L])
ax1.set_title(f'Buoyancy θ at t={state["time"]:.2f}')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
plt.colorbar(im, ax=ax1)

# Energy evolution
ax2.plot(times, energies, 'b-', linewidth=2)
ax2.set_xlabel('Time')
ax2.set_ylabel('Total Energy')
ax2.set_title('Energy Evolution')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\nSimulation summary:")
print(f"  Final time: {state['time']:.3f}")
print(f"  Energy change: {(energies[-1]/energies[0] - 1)*100:.2f}%")
print(f"  θ range: [{theta.min():.3f}, {theta.max():.3f}]")

## 6. Energy Spectrum

The energy spectrum reveals the scale-by-scale distribution of energy:

In [ ]:
from pygsquig.utils.diagnostics import compute_energy_spectrum

# Compute spectrum
k_bins, E_k = compute_energy_spectrum(state['theta_hat'], grid, alpha)

# Plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.loglog(k_bins, E_k, 'b-', linewidth=2, label='Simulation')

# Reference slopes
k_ref = k_bins[k_bins > 10]
if len(k_ref) > 0:
    # k^{-5/3} scaling
    E_ref = E_k[k_bins == k_ref[0]][0] * (k_ref/k_ref[0])**(-5/3)
    ax.loglog(k_ref, E_ref, 'k--', alpha=0.5, label='k⁻⁵/³')

ax.set_xlabel('Wavenumber k')
ax.set_ylabel('Energy Spectrum E(k)')
ax.set_title('Energy Spectrum')
ax.grid(True, alpha=0.3, which='both')
ax.legend()
ax.set_xlim(1, N/2)

plt.show()

## 7. Performance Benchmark

Let's measure the computational performance:

In [ ]:
import time

# Create a fresh state
state_bench = solver.initialize(seed=123)

# Warmup (for JIT compilation)
for _ in range(5):
    state_bench = solver.step(state_bench, dt)

# Time 100 steps
start = time.perf_counter()
for _ in range(100):
    state_bench = solver.step(state_bench, dt)
elapsed = time.perf_counter() - start

steps_per_second = 100 / elapsed
ms_per_step = elapsed / 100 * 1000

print(f"Performance (N={N}):")
print(f"  Time per step: {ms_per_step:.2f} ms")
print(f"  Steps per second: {steps_per_second:.1f}")
print(f"  Device: {jax.default_backend()}")

## 8. Common Issues and Troubleshooting

### Issue 1: NaN values appearing immediately

In [ ]:
# WRONG: Hyperviscosity too large
solver_bad = gSQGSolver(grid, alpha=1.0, nu_p=1e-2, p=8)  # ❌ Too large!
state_bad = solver_bad.initialize(seed=42)

# This will produce NaN
state_bad = solver_bad.step(state_bad, dt)
E_bad = compute_total_energy(state_bad['theta_hat'], grid, alpha)
print(f"Energy with ν_p=1e-2: {E_bad} (NaN!)")

# CORRECT: Appropriate hyperviscosity
solver_good = gSQGSolver(grid, alpha=1.0, nu_p=1e-16, p=8)  # ✓ Correct
state_good = solver_good.initialize(seed=42)
state_good = solver_good.step(state_good, dt)
E_good = compute_total_energy(state_good['theta_hat'], grid, alpha)
print(f"Energy with ν_p=1e-16: {E_good:.6f} (physical!)")

### Issue 2: Time step too large

In [ ]:
# Check CFL condition
from pygsquig.core.operators import compute_velocity_from_theta

# Compute velocity
u, v = compute_velocity_from_theta(state['theta_hat'], grid, alpha)
max_vel = float(jnp.max(jnp.sqrt(u**2 + v**2)))

# CFL condition
dx = L / N
dt_cfl = 0.5 * dx / max_vel  # Safety factor of 0.5

print(f"CFL analysis:")
print(f"  Max velocity: {max_vel:.3f}")
print(f"  Grid spacing: {dx:.3f}")
print(f"  CFL limit: dt < {dt_cfl:.4f}")
print(f"  Current dt: {dt:.4f}")
print(f"  Status: {'✓ Safe' if dt < dt_cfl else '❌ Too large!'}")

### Issue 3: Checking numerical stability

In [ ]:
def check_stability(state, solver, dt, n_steps=10):
    """Quick stability check."""
    state_test = state.copy()
    
    for i in range(n_steps):
        state_test = solver.step(state_test, dt)
        theta_hat = state_test['theta_hat']
        
        # Check for issues
        has_nan = jnp.any(jnp.isnan(theta_hat))
        has_inf = jnp.any(jnp.isinf(theta_hat))
        max_val = jnp.max(jnp.abs(theta_hat))
        
        if has_nan or has_inf or max_val > 1e10:
            print(f"❌ Instability detected at step {i+1}:")
            print(f"   NaN: {has_nan}, Inf: {has_inf}, Max: {max_val:.2e}")
            return False
    
    print(f"✓ Stable for {n_steps} steps")
    return True

# Test current configuration
check_stability(state, solver, dt)

## Summary

You've successfully:
1. ✓ Verified your pygSQuiG installation
2. ✓ Created a computational grid and solver
3. ✓ Run a decaying turbulence simulation
4. ✓ Computed energy spectra and diagnostics
5. ✓ Learned to troubleshoot common issues

### Key takeaways:
- Always use appropriate hyperviscosity values (ν_p ~ 1e-16 for p=8)
- Monitor energy conservation and check for NaN values
- Respect CFL conditions for time stepping
- Use powers of 2 for grid resolution (FFT efficiency)

### Next steps:
- Try different values of α (0.5, 1.5, 2.0)
- Explore forced turbulence (see notebook 03)
- Add passive scalars (see notebook 05)
- Use adaptive timestepping (see notebook 04)